FITBIR PV-UMLS CODE MATCHING


<br>
This script queries the UMLS metathesaurus api OR MetaMap API for CUIs related to data element permissible value descriptions (PVDs) <br>
DESCRIPTION:<br>
    <br>
(1): Connect to UMLS api OR MetaMap API<br>
(2): Takes dataElementExport.csv file and explodes 'permissible value description' (PVD) and 'permissible values' columns (one PV/PVD per row)<br>
(3): Cycles through each exploded PVD and queries with PVD (or PV if PVD not available). 'Exact' search type first, but if there are no results, then 'normalizedWords' search type. <br>
(4): Creates file for user to MANUALLY mark rows to be filtered out in the subsequent script "PV_UMLS_FITBIR_DE_REPORT_merge.py"<br>
EXCEL WORKBOOK CONTENTS:<br>
    <br>
    - sheet1: Exploded PVDs/PVs with UMLS codes and remove column --> df_PVDs_UMLS<br>
    - sheet2: Original Data Element report --> df_DE_report<br>
    - sheet3: Data Element report with PVDs exploded -->  df_DE_report_PVDs_exploded<br>


LOAD PACKAGES

In [1]:
import cui_mapping_functions as cmf # contains custom functions for loading/processing FITBIR csv datafiles
import requests
import json
import pandas as pd
import numpy as np
import math
import os
import importlib
import tkinter as tk  # for dialog boxes
from skr_web_api import Submission
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
importlib.reload(cmf)

<module 'cui_mapping_functions' from 'C:\\Users\\armengolkm\\Documents\\BRICS\\Sourcetree_BRICS\\Operations\\UMLS Coding\\UMLS-CUI_DE-PV_pipeline_ipynb_Version-1.5_2022-12-06\\cui_mapping_functions.py'>

MASTER SETTINGS

In [2]:
api = 'UMLS' #'MetaMap'  # use MetaMap to search for CUIs / # api = 'UMLS'  # search UMLS thesaurus directly for CUIs
rmv_stopwords_1_0 = 1
use_cheatsheet = 0
source_filter = None  # source_filter = 'NCI'

API CHOICE AND INTIALIZATION: UMLS or METAMAP

In [3]:
if api == 'UMLS':

    # UMLS API AUTHENTICATION
    apiKey = 'd507799b-ddc3-4592-b7b5-287fe6d3bf7c'
    url = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
    payload = f"apikey={apiKey}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)

    # UMLS SEARCH BY TERM INITALiZATION
    uri = "https://uts-ws.nlm.nih.gov"
    version = 'current'  # include version number in output for reproducibilty
    content_endpoint = "/rest/search/"+version
    payload = {}
    headers = {}
    searchType1 = 'exact'
    searchType2 = 'normalizedWords'  # searchType2 = 'approximate' # now deprecated
    pageSize = 25  # default is 25 results per page
    pages_max = 1
elif api == 'MetaMap':

    # METAMAP API AUTHENTICATION
    email = 'armengolkm@nih.gov'
    apikey = 'd507799b-ddc3-4592-b7b5-287fe6d3bf7c'
    serverurl = "https://utslogin.nlm.nih.gov/cas/v1/tickets"
    tgtserverurl = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
    serviceurl = 'https://ii.nlm.nih.gov/cgi-bin/II/UTS_Required/API_batchValidationII.pl'

    # METAMAP SEARCH SETTINGS
    cmd = 'metamap'
    mm_data_year = '-Z 2020AB'
    mm_data_version = '-V USAbase'
    relaxed_model = '-C'
    show_candidates = '-c'
    show_cuis = '-I'
    ignore_word_order = '-i'
    sources = '-G'
    term_processing = '-z'
    word_sense_disambiguation = '-y'
    formatted_json_output = '--JSONf 2'
    cmdargs = ' '.join([mm_data_year, mm_data_version, relaxed_model, show_candidates, show_cuis, ignore_word_order, sources,
                        term_processing,  word_sense_disambiguation, formatted_json_output])


    <!DOCTYPE HTML PUBLIC \"-//IETF//DTD HTML 2.0//EN\"><html lang="ENG"><head><title>201 Created</title></head><body>
    <h1>TGT Created</h1><form action="https://utslogin.nlm.nih.gov/cas/v1/api-key/TGT-41920-1emw14e1lbcmr747-cas" method="POST">Service:
    <input type="text" name="service" value=""><br>
    <input type="submit" value="Submit"></form></body></html>
    


CHOOSE dataElementExport.csv FILE

In [4]:
fp_DE_report = filedialog.askopenfilename(
    title='Select dataElementExport.csv file')  # choose dataElementExport.csv file
df_DE_report = pd.read_csv(fp_DE_report)
dir_DE_report = os.path.dirname(fp_DE_report)
fn_DE_report_new = os.path.basename(dir_DE_report)
fs_nam = fn_DE_report_new.split('_')[0]

EXPLODE ROWS OF DATA ELEMENT REPORTS

In [5]:
dir_step1 = cmf.create_folder(
    dir_DE_report + f"/PV_Step-1_Explode PVDs and search CUI_{api}")
df_DE_report_temp = df_DE_report.dropna(axis=0, how='all', subset=[
                                        'permissible values', 'permissible value descriptions']).copy()  # removes variables with no PVs or PVDs
df_DE_report_temp['PVs_exploded'] = df_DE_report_temp['permissible values'].str.split(
    '|')  # turns pipe-separated values into list for explode step
df_DE_report_temp['PVDs_exploded'] = df_DE_report_temp['permissible value descriptions'].str.split(
    '|')  # turns pipe-separated  separated values into list for explode step
df_DE_report_PVDs_exploded = df_DE_report_temp.explode(
    ['PVs_exploded', 'PVDs_exploded'], ignore_index=True)  # explode PVs/PVDs

TERM PROCESSING

In [6]:
df_PVD_processed = df_DE_report_PVDs_exploded[['variable name', 'title', 'definition', 'preferred question text', 'domain.general (for all diseases)',
                                               'PVs_exploded', 'PVDs_exploded']].copy()

In [7]:
pv_no_punct = df_PVD_processed['PVs_exploded'].apply(
    cmf.remove_punctuation).squeeze()  # remove punctuation
pvd_no_punct = df_PVD_processed['PVDs_exploded'].apply(
    cmf.remove_punctuation).squeeze()  # remove punctuation

In [9]:
if rmv_stopwords_1_0:
    # choose csv file with list of stopwords Olga curated
    fp_stopwords = filedialog.askopenfilename(
        title='Select MetaMap_Settings_StopWords.csv file')
    df_stopwords = pd.read_csv(fp_stopwords)
    # will remove in future version of stopwords file
    # df_stopwords = df_stopwords[df_stopwords['StopWord?'] == 'Yes']
    # make lower case and remove leading/trailing whitespaces
    mm_stopwords = list(df_stopwords['Word'].str.lower().str.strip())
    df_PVD_processed[['PV_query_term', 'PV_stopwords_removed']] = pv_no_punct.apply(
        cmf.remove_stopwords, mm_stopwords=mm_stopwords)  # remove stopwords
    df_PVD_processed[['PVD_query_term', 'PVD_stopwords_removed']] = pvd_no_punct.apply(
        cmf.remove_stopwords, mm_stopwords=mm_stopwords)
else:
    df_PVD_processed['PV_query_term'] = pv_no_punct
    df_PVD_processed['PVD_query_term'] = pvd_no_punct
    df_PVD_processed[['PV_stopwords_removed',
                      'PVD_stopwords_removed']] = ''  # remove stopwords
if use_cheatsheet:
    # choose csv file with list of stopwords Olga curated
    fp_cheatsheet = filedialog.askopenfilename(
        title='Select MetaMap_Settings_Cheatsheet.csv file')
    df_cheatsheet = pd.read_csv(fp_cheatsheet)

METAMAP OR UMLS API LOOP

In [10]:
if api == 'MetaMap':

    # FORMAT FITBIR DATA ELEMENTS FOR METAMAP INGESTION 
    
    pvd_empty = df_PVD_processed[df_PVD_processed['PVD_query_term'] == ''].index
    pvd_present = df_PVD_processed.loc[~df_PVD_processed.index.isin(
        pvd_empty)].index
    df_PVD_processed.loc[pvd_present,  'PV_or_PVD_search'] = 'PVD_query_term'
    df_PVD_processed.loc[pvd_empty,  'PV_or_PVD_search'] = 'PV_query_term'
    series_temp = pd.Series(index=df_PVD_processed.index, dtype='object')
    series_temp.loc[pvd_present] = df_PVD_processed.loc[pvd_present,
                                                        'PVD_query_term']
    series_temp.loc[pvd_empty] = df_PVD_processed.loc[pvd_empty,
                                                      'PV_query_term']
    df_PVD_processed['MetaMap_query_term'] = series_temp
    df_PVD_processed['MetaMap_PMID'] = df_PVD_processed[[
        'variable name', 'MetaMap_query_term']].agg('_'.join, axis=1)  # create MetaMap input
    df_PVD_processed['MetaMap_input'] = df_PVD_processed[[
        'MetaMap_PMID', 'MetaMap_query_term']].agg('|'.join, axis=1)  # create MetaMap input
    fp_df_PVD_processed = f"{dir_step1}/PV_step1_{fs_nam}_MM_processed.xlsx"
    # file you will have to manually review and specify whether to keep a row
    writer = pd.ExcelWriter(fp_df_PVD_processed, engine='openpyxl')
    sheet1 = 'MM_PV_PROCESSING'
    df_PVD_processed.to_excel(writer, sheet_name=sheet1, index=False)
    writer.close()
    df_mm_inputfile = df_PVD_processed['MetaMap_input']
    fp_df_mm_inputfile = f"{dir_step1}/PV_step1_{fs_nam}_MM_inputfile_SingLinePMID.txt"
    with open(fp_df_mm_inputfile, 'w') as f:
        for cell in df_mm_inputfile:
            f.write(cell)
            f.write("\n")
    inputfile = fp_df_mm_inputfile
    inst = Submission(email, apikey)
    inst.set_serviceurl(serviceurl)
    inst.init_generic_batch(cmd, cmf.unescape_string(cmdargs))
    inst.set_batch_file(inputfile)
    inst.form['SingLinePMID'] = 'yes'
    inst.form['Batch_Command'] = '{} {}'.format(
        cmd, cmf.unescape_string(cmdargs))
    response = inst.submit()
    print('response status: {}'.format(response.status_code))
    print('content: {}'.format(response.content.decode()))
    if response.status_code == 200:
        jsondata0 = response.content.decode()
        print('content: {}'.format(jsondata0))
        # remove string "NOT DONE LOOP" if present
        jsondata = jsondata0.replace('NOT DONE LOOP', '')
        data = json.loads(jsondata)
        print('data:{}'.format(data))
        fp_json_output = f"{dir_step1}/PV_step1_{fs_nam}_MM_json_output.json"
        with open(fp_json_output, 'w') as chan:
            chan.write(jsondata)

    # PROCESS METAMAP OUTPUT FOR MANUAL REVIEW AND CURATION
    dict_docs = data["AllDocuments"]
    ls_df_maps = []
    cols_map = ['CandidateScore', 'CandidateCUI', 'CandidateMatched',
                'CandidatePreferred', 'MatchedWords', 'SemTypes', 'MatchMaps', 'IsHead',
                'IsOverMatch', 'Sources', 'ConceptPIs', 'Status', 'Negated']  # fields to extract for review
    for doc in dict_docs:
        pmid = doc['Document']['Utterances'][0]['PMID']
        df_temp = pd.json_normalize(doc['Document']['Utterances'][0]['Phrases'], record_path=[
                                    'Candidates'], errors='ignore')
        if df_temp.empty:
            df_temp = pd.DataFrame(columns=cols_map)
        df_temp.insert(0, 'PMID', pmid)
        ls_df_maps.append(df_temp)

    # combine separate dataframes for each PMID into one dataframe
    df_maps = pd.concat(ls_df_maps)
    df_maps['keep'] = np.nan
    df_final = pd.merge(df_PVD_processed, df_maps,
                        left_on='MetaMap_PMID', right_on='PMID', how='inner', suffixes=('', '_y'))  # adds data element metadata useful for review
    df_final[['CandidateCUI', 'CandidateMatched', 'CandidatePreferred', 'Sources']] = df_final[[
        'CandidateCUI', 'CandidateMatched', 'CandidatePreferred', 'Sources']].fillna('Not Available')
    df_final.rename(columns={'CandidateCUI': "permissible value concept identifiers",
                             'CandidatePreferred': 'permissible value concept names', 'Sources': 'permissible value terminology sources'}, inplace=True)
    df_final.drop(df_maps.filter(regex='_y$').columns, axis=1, inplace=True)
    fp_step1 = f"{dir_step1}/PV_step1_{fs_nam}_MM_keepCol.xlsx"
    writer = pd.ExcelWriter(fp_step1, engine='openpyxl')
    sheet1 = 'PVDs_MetaMap'
    sheet2 = 'DE_report'
    sheet3 = 'DE_report_PVDs_exploded'
    idx_golden = cmf.use_cheatsheet(df_final, df_cheatsheet)
    df_final.loc[idx_golden, 'keep'] = 'x'
    df_final.to_excel(writer, sheet_name=sheet1, index=False)
    df_DE_report.to_excel(writer, sheet_name=sheet2, index=False)
    df_DE_report_PVDs_exploded.to_excel(writer, sheet_name=sheet3, index=False)
    ws1 = writer.sheets[sheet1]
    for i, width in enumerate(cmf.get_header_widths(df_final)):
        excel_col_letter = cmf.get_column_letter(i+1)
        ws1.column_dimensions[excel_col_letter].width = width
    cols_hide = ['domain.general (for all diseases)', 'PV_processed', 'PVD_processed', 'PV_query_term', 'PV_stopwords_removed', 'PVD_query_term', 'PVD_stopwords_removed', 'PMID',
                 'MatchMaps',  'ConceptPIs']
    cols_hide_idx = [df_final.columns.get_loc(
        c) for c in cols_hide if c in df_final]
    for col_idx in cols_hide_idx:
        col_letter_hide = cmf.get_column_letter(col_idx + 1)
        # these columns are hidden as they aren't useful for review
        ws1.column_dimensions[col_letter_hide].hidden = True
    writer.close()
elif api == 'UMLS':

    # QUERY UMLS API WITH PVD OR PV TERM
    df_PVDs_UMLS = pd.DataFrame(columns=['count', 'variable name', 'PV_query_term', 'PVD_query_term',
                                         'PV_or_PVD_search', 'searchType', 'result',
                                         'permissible value terminology sources', 'permissible value concept names',
                                         'permissible value concept identifiers', 'url'])
    columnSeriesObj = zip(df_PVD_processed['variable name'],
                          df_PVD_processed['PV_query_term'],
                          df_PVD_processed['PVD_query_term'])
    cnt_searchTerm = 0
    idx_row = -1
    for vn, pv_qt, pvd_qt in columnSeriesObj:
        ''' This loop incrimentally appends new search values to the text string'''
        pageNumber = 1
        cnt_searchTerm += 1
        idx_row += 1
        pv_qt = pv_qt.strip()  # in case an empty value actually has spaces in it
        pvd_qt = pvd_qt.strip()  # in case an empty value actually has spaces in it
        searchTerm = pvd_qt  # default search term is PVD
        searchTermType = 'PVD search'
        if pd.isna(searchTerm) or len(searchTerm) == 0:
            df_PVDs_UMLS.loc[idx_row] = ['0', vn, pv_qt, pvd_qt, searchTermType,
                                         'NONE', 'No search. Value is nan or empty.', '', '', '', '']
            print(
                f"{searchTermType} ({cnt_searchTerm}) : '{searchTerm}' is nan or empty. Trying PV.")
            searchTermType = 'PV search'
            searchTerm = pv_qt
            if pd.isna(searchTerm) or len(searchTerm) == 0:
                df_PVDs_UMLS.loc[idx_row] = ['0', vn, pv_qt, pvd_qt, searchTermType,
                                             'NONE', 'No search. Value is nan or empty.', '', '', '', '']
                print(
                    f"{searchTermType} ({cnt_searchTerm}) : '{searchTerm}' is nan or empty. Moving on to next PVD.")
                continue
        query = {'apiKey': apiKey, 'string': searchTerm, 'searchType': searchType1,
                 'pageSize': pageSize, 'pageNumber': pageNumber}  # searchType = exact match to get rid of excessive results
        #query['includeObsolete'] = 'true' # return content that is a result of matches on obsolete terms.
        #query['includeSuppressible'] = 'true' # return content that is a result of matches on suppressible terms.
        #query['returnIdType'] = "sourceConcept" # specifies the type of identifier you wish to retrieve.
        #query['sabs'] = "SNOMEDCT_US" # comma-separated list of source vocabularies to include in your searc
        r = requests.get(uri + content_endpoint, params=query)
        r.encoding = 'utf-8'
        items = json.loads(r.text)
        jsonData = items["result"]
        recCount = jsonData['recCount']
        if not recCount:  # for cases where the 'exact' search type results in an empty list
            print(
                f"{searchTermType} ({cnt_searchTerm}): '{searchTerm}' has no exact match. Trying alternative searchType.")
            query = {'apiKey': apiKey, 'string': searchTerm, 'searchType': searchType2,
                     'pageSize': pageSize, 'pageNumber': pageNumber}  # searchType = approximate
            r = requests.get(uri + content_endpoint, params=query)
            r.encoding = 'utf-8'
            items = json.loads(r.text)
            jsonData = items["result"]
            recCount = jsonData['recCount']
            if not recCount:  # if approximate search still results in nothing
                print(
                    f"{searchTermType} ({cnt_searchTerm}): '{searchTerm}' has no alternative searchType match. Moving on to next PVD.")
                df_PVDs_UMLS.loc[idx_row] = ['0', vn, pv_qt, pvd_qt, searchTermType,
                                             searchType2, 'No approximate match.', '', '', '', '']
                continue
            else:
                print(
                    f"{searchTermType} ({cnt_searchTerm}): '{searchTerm}' has {recCount} alternative searchType matches.")
                pages_idx = list(range(1, math.ceil(recCount/pageSize)+1))
                # loop through pages limited by specified pages_max
                pages_to_view = [
                    page for page in pages_idx if page <= pages_max]
                for pg in pages_to_view:
                    if pg != 1:
                        query = {'apiKey': apiKey, 'string': searchTerm, 'searchType': searchType2,
                                 'pageSize': pageSize, 'pageNumber': pg}  # searchType = approximate
                        r = requests.get(uri + content_endpoint, params=query)
                        r.encoding = 'utf-8'
                        items = json.loads(r.text)
                        jsonData = items["result"]
                    for result in jsonData["results"]:
                        idx_row += 1
                        try:
                            df_PVDs_UMLS.loc[idx_row] = [str(recCount), vn, pv_qt, pvd_qt, searchTermType, searchType2,
                                                         'Approximate match.', result[
                                                             'rootSource'], result["name"], result["ui"],
                                                         "https://uts.nlm.nih.gov/uts/umls/concept/" + result["ui"]]
                        except NameError:
                          print("NameError occurred. Some variable isn't defined.")
        else:
            print(
                f"{searchTermType} ({cnt_searchTerm}): '{searchTerm}' has {recCount} exact searchType matches.")
            pages_idx = list(range(1, math.ceil(recCount/pageSize)+1))
            # loop through pages limited by specified pages_max
            pages_to_view = [page for page in pages_idx if page <= pages_max]
            for pg in pages_to_view:
                if pg != 1:
                    query = {'apiKey': apiKey, 'string': searchTerm, 'searchType': searchType2,
                             'pageSize': pageSize, 'pageNumber': pg}  # searchType = approximate
                    r = requests.get(uri + content_endpoint, params=query)
                    r.encoding = 'utf-8'
                    items = json.loads(r.text)
                    jsonData = items["result"]
            for result in jsonData["results"]:
                idx_row += 1
                try:
                    df_PVDs_UMLS.loc[idx_row] = [str(len(jsonData["results"])), vn, pv_qt, pvd_qt, searchTermType, searchType1,
                                                 'Exact match.', result['rootSource'], result["name"], result["ui"],
                                                 "https://uts.nlm.nih.gov/uts/umls/concept/" + result["ui"]]
                except NameError:
                  print("NameError occurred. Some variable isn't defined.")

    # REFORMAT OUTPUT TO DATAFRAME AND EXCEL WORKBOOK
    df_final = pd.merge(df_PVDs_UMLS, df_PVD_processed,
                        left_on=['variable name',
                                 'PV_query_term', 'PVD_query_term'],
                        right_on=['variable name',
                                  'PV_query_term', 'PVD_query_term'],
                        how='outer', suffixes=('', '_y'))
    df_final.drop(df_final.filter(regex='_y$').columns, axis=1, inplace=True)
    col_reorder = ['count', 'variable name', 'title', 'definition', 'preferred question text', 'domain.general (for all diseases)',
                   'PVs_exploded', 'PVDs_exploded', 'PV_query_term', 'PVD_query_term',
                   'PV_stopwords_removed', 'PVD_stopwords_removed',
                   'PV_or_PVD_search', 'searchType', 'result',
                                           'permissible value terminology sources', 'permissible value concept names',
                                           'permissible value concept identifiers', 'url']
    df_final = df_final[col_reorder]
    df_final['keep'] = np.nan
    fp_step1 = f"{dir_step1}/PV_step1_{fs_nam}_UMLS_keepCol.xlsx"
    # files you will have to manually review and specify whether to keep row
    writer = pd.ExcelWriter(fp_step1, engine='openpyxl')
    sheet1 = 'PVDs_UMLS'
    sheet2 = 'DE_report'
    sheet3 = 'DE_report_PVDs_exploded'
    if use_cheatsheet:
        idx_golden = cmf.use_cheatsheet(df_final, df_cheatsheet)
        df_final.loc[idx_golden, 'keep'] = 'x'
    df_final.to_excel(writer, sheet_name=sheet1, index=False)
    ws1 = writer.sheets[sheet1]
    for i, width in enumerate(cmf.get_header_widths(df_final)):
        excel_col_letter = cmf.get_column_letter(i+1)
        ws1.column_dimensions[excel_col_letter].width = width
    cols_hide = ['domain.general (for all diseases)', 'PVs_exploded', 'PVDs_exploded',
                 'PV_stopwords_removed', 'PVD_stopwords_removed', 'url']
    cols_hide_idx = [df_final.columns.get_loc(
        c) for c in cols_hide if c in df_final]
    for col_idx in cols_hide_idx:
        col_letter_hide = cmf.get_column_letter(col_idx + 1)
        # these columns are hidden as they aren't useful for review
        ws1.column_dimensions[col_letter_hide].hidden = True
    df_DE_report.to_excel(writer, sheet_name=sheet2, index=False)
    df_DE_report_PVDs_exploded.to_excel(writer, sheet_name=sheet3, index=False)
    writer.close()

PVD search (1) : '' is nan or empty. Trying PV.
PV search (1): 'case' has 4 exact searchType matches.
PVD search (2): 'control' has 9 exact searchType matches.
PVD search (3): 'unknown' has 38 exact searchType matches.
PVD search (4): 'brother' has 3 exact searchType matches.
PVD search (5): 'chartmedical record' has no exact match. Trying alternative searchType.
PVD search (5): 'chartmedical record' has no alternative searchType match. Moving on to next PVD.
PVD search (6): 'daughter' has 1 exact searchType matches.
PVD search (7): 'family relation' has 1 exact searchType matches.
PVD search (8): 'father' has 4 exact searchType matches.
PVD search (9): 'friend' has 2 exact searchType matches.
PVD search (10): 'mother' has 2 exact searchType matches.
PVD search (11): 'other' has 44 exact searchType matches.
PVD search (12): 'participantsubject' has no exact match. Trying alternative searchType.
PVD search (12): 'participantsubject' has no alternative searchType match. Moving on to next